# Indeed Job Scraper
create a general purpose job scraper

In [1]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

### Setup the query and url

In [2]:
def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

In [3]:
url = get_url('senior accountant', 'charlotte nc')
print(url)

https://www.indeed.com/jobs?q=senior+accountant&l=charlotte+nc


### Extract the html data

In [4]:
response = requests.get(url)

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
cards = soup.find_all('div', 'jobsearch-SerpJobCard')

### Prototype the model with a single record

In [7]:
card = cards[0]

In [8]:
job_title = card.h2.a.get('title')

In [9]:
company = card.find('span', 'company').text.strip()

In [10]:
job_location = card.find('div', 'recJobLoc').get('data-rc-loc')

In [16]:
post_date = card.find('span', 'date').text
today = datetime.today().strftime('%Y-%m-%d')

In [12]:
summary = card.find('div', 'summary').text.strip().replace('\n', ' ')

In [13]:
# this does not exists for all jobs, so handle the exceptions
salary_tag = card.find('span', 'salaryText')
if salary_tag:
    salary = salary_tag.text.strip()
else:
    salary = ''    

In [14]:
job_url = 'https://www.indeed.com' + card.h2.a.get('href')

In [17]:
record = (job_title, company, job_location, post_date, today, summary, salary, job_url)

In [18]:
record

('Senior Accountant',
 'Commercial Credit Group',
 'Charlotte, NC',
 '30+ days ago',
 '2020-09-03',
 'The Senior Accountant ensures the integrity of accounting data by recording, verifying, and consolidating transactions for a Commercial Finance business.',
 '',
 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Ckbjaw4Q0dRT0YLc9RYl82KG8jKHfeJJWK1M-sDYTDPxoG8b9ZQRqgphJYWKY9bEqmxOXW86GlerA1XeXf0jmMUQNt2LxL9eFGmsRUeFM-bA6T6NgARY_W27r5Hz3QEwC5OWIXERHGcV2NugqPN2whIAgoYB8gjCU2EpwhokhCI2SeRe2cwzGf34Q8Lxqd2rdClj3Fsw8DEPd4x3skF9jHVybqYAOuNwfp4Sc9nevbzo0-rlQM7mbFbS1ettrarJrrCOv_EnDSuWKaEnpN_fKntX0iwgmQvfVVkOpmfy5MsMWLcgleQWz7gDaWAWbrTbqlMCp3fgYVScOrXuL2JGY20UYwDg2oQ--uAnOtREGY43fGdrgWk8IwJ2h1sWTGF05VNcmsheUGNa7--dtBC9bdA7k2h0Qlk-t6GisH_zSJwxL9iFVODN_UTlD2jzKQUSrAWzyOLCt24jL0mi0wYlYT&p=0&fvj=1&vjs=3')

### Generalize the model with a function

In [19]:
def get_record(card):
    """Extract job data from a single record"""
    
    job_title = card.h2.a.get('title')
    company = card.find('span', 'company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'summary').text.strip().replace('\n', ' ')
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')

    # this does not exists for all jobs, so handle the exceptions
    salary_tag = card.find('span', 'salaryText')
    if salary_tag:
        salary = salary_tag.text.strip()
    else:
        salary = ''  
        
    record = (job_title, company, job_location, post_date, today, summary, salary, job_url)
    return record

In [20]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

### Get the next page

In [25]:
while True:
    try:
        url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
    except AttributeError:
        break

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'jobsearch-SerpJobCard')

    for card in cards:
        record = get_record(card)
        records.append(record)

### Putting it all together

In [26]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup


def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url


def get_record(card):
    """Extract job data from a single record"""
    
    job_title = card.h2.a.get('title')
    company = card.find('span', 'company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'summary').text.strip().replace('\n', ' ')
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')

    # this does not exists for all jobs, so handle the exceptions
    salary_tag = card.find('span', 'salaryText')
    if salary_tag:
        salary = salary_tag.text.strip()
    else:
        salary = ''  
        
    record = (job_title, company, job_location, post_date, today, summary, salary, job_url)
    return record


def main(position, location):
    """Run the main program routine"""
    records = []
    url = get_url(position, location)
    
    # extract the job data
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')
        for card in cards:
            record = get_record(card)
            records.append(record)
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError:
            break
        
    # save the job data
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'Salary', 'JobUrl'])
        writer.writerows(records)

In [27]:
# run the main program
main('senior accountant', 'charlotte nc')